In [1]:
from random import randint, uniform, choice
from util import generate_problem, verify_witness

INF = 1e9

## Firstly we need a solver for the STP problem.

note here we are given a distance graph
(the generation is done in the genetic part, so we can reuse memory and save time when an interval selection is changed.)

In [2]:
def discrete_graph(N):
    mat = [[INF for _ in range(N)] for _ in range(N)]
    for i in range(N):
        mat[i][i] = 0
    return mat

def generate_d_graph(graph):
    N = len(graph)
    E = discrete_graph(N)
    
    for i in range(N):
        E[i][i] = 0 # redundant

    for i in range(N):
        for j in range(N):
            E[i][j] = graph[i][j]

    for k in range(N):
        for i in range(N):
            for j in range(N):
                E[i][j] = min(E[i][j], E[i][k] + E[k][j])

    return E

def get_min_sol(d_graph):
    return [-d_graph[i][0] for i in range(len(d_graph))]

def consistent(d_graph):
    for i in range(len(d_graph)):
        if d_graph[i][i] < 0: return False
    return True

def solve_stp(graph):
    ''' graph is the constraint adjacency matrix '''
    
    d_graph = generate_d_graph(graph)
    
    return consistent(d_graph), get_min_sol(d_graph)

## The genetic part:

In [21]:
def update_graph(gene):
    # set up graph using index 0 everywhere
    # so later we can change interval in O(1) time
    # (saves memory and time, and this same approach is especially beneficial in meta_walk)
    
    interval_selection = gene[0]
    graph = gene[1]
    gene[2] = None # unset
    
    for i, constr in enumerate(T):
        interval_index = interval_selection[i]
        
        i, j = constr['i'], constr['j']
        interval = constr['intervals'][interval_index]
        
        graph[i][j] = interval[1]
        graph[j][i] = -interval[0]

def update_graph_at_constraint(gene, constraint_index):
    ''' update_graph but updates only one constraint '''
    
    interval_selection = gene[0]
    graph = gene[1]
    gene[2] = None # unset
    
    interval_index = interval_selection[constraint_index]
    constr = T[constraint_index]
    
    i, j = constr['i'], constr['j']
    interval = constr['intervals'][interval_index]

    graph[i][j] = interval[1]
    graph[j][i] = -interval[0]

def random_gene(T):
    num_variables = max([max(t['i'], t['j']) for t in T])
    interval_selection = [randint(0, len(constr['intervals'])-1) for constr in T]
    gene = [interval_selection, discrete_graph(num_variables+1), None]
    update_graph(gene)
    d_graph = generate_d_graph(gene[1]) # costly op.
    failed = verify_witness(get_min_sol(d_graph), T)
    gene[2] = failed
    
    return gene
    
def walk_gene(gene, T):
    # ignore all that have no choice
    # create list of plausible indices and filter it.
    candidate_indices = [j for j in range(len(T))]
    candidate_indices = list(filter(
        lambda x: len(T[x]['intervals']) > 1,
        candidate_indices
    ))
    
    idx = choice(candidate_indices)
    gene[0][idx] = randint(0, len(T[idx]['intervals'])-1)
    update_graph_at_constraint(gene, idx)
    
def fitness(gene, T):
    # make sure that gene[2] is set to failed constraints
    # otherwise this will not work ideally (or fail if it's None.)
    return -len(gene[2])

def select(genes, retainment_ratio, T):
    genes.sort(key=lambda g: -fitness(g, T))
    return genes[: int(len(genes)*retainment_ratio+1) ]






def crossover(genes, gene_pool_size):
    raise 'todo'
'''    while len(genes) < gene_pool_size:
        i = randint(0, len(genes)-1)
        j = randint(0, len(genes)-1)
        cross_index = randint(1, len(genes[0])-1)
        genes += [ genes[i][:cross_index] + genes[j][cross_index:] ]
        
    return genes'''

def mutate(T, genes, mutation_chance, r):
    for g in genes:
        if uniform(0, 1) < mutation_chance:
            walk_gene(g, T)
            
    return genes

def meta_genetic(T, r,
            gene_pool_size,
            retainment_ratio,
            mutation_chance,
            max_iterations):
    
    genes = [random_gene(T) for i in range(gene_pool_size)]
    
    for it in range(max_iterations):
        genes = select(genes, retainment_ratio, T)
        genes = crossover(genes, gene_pool_size)
        genes = mutate(T, genes, mutation_chance, r)
    
    best_gene = select(genes, 1, T)[0] # to sort genes such that first is best
    best_gene_failed = verify_witness(best_gene, T)
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)


def asddddddddddddddddd(T, max_iterations, max_flips):
    num_variables = max([max(t['i'], t['j']) for t in T])
    graph = discrete_graph(num_variables+1)
    
    best_gene = None
    best_gene_failed = None
    
    for i in range(max_iterations):
        gene = random_gene(T)
        graph = update_graph(graph, gene)
        
        for j in range(max_flips):
            # at each iteration we modify

            is_consistent, witness = solve_stp(graph)
            gene_failed = verify_witness(witness, T)

            if not best_gene or len(gene_failed) < len(best_gene_failed):
                best_gene = gene
                best_gene_failed = gene_failed

            if is_consistent:
                break
                
            gene = walk_gene(gene, T)
        
    print(f'num iterations: {i+1}')
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)

In [17]:
T = generate_problem()
gene = random_gene(T)
gene

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0],
 [[100, 100, 100, 100, 100, 100, 100, 100, 100, 100],
  [100,
   0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   -35],
  [100,
   1000000000.0,
   0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   45,
   1000000000.0],
  [100,
   1000000000.0,
   1000000000.0,
   0,
   1000000000.0,
   -74,
   43,
   61,
   1000000000.0,
   1000000000.0],
  [100,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0],
  [100,
   1000000000.0,
   1000000000.0,
   87,
   1000000000.0,
   0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0],
  [100,
   1000000000.0,
   1000000000.0,
   -25,
   1000000000.0,
   1000000000.0,
   0,
   1000000000.0,
   80,
   1000000000.0],
  [100,
   1000000000.0,
   1000000000.0,
   33,
   1000000000.

In [20]:
walk_gene(gene, T)

UnboundLocalError: local variable 'i' referenced before assignment